In [99]:
import pandas as pd
import numpy as np
import requests
import datetime as dt
from bs4 import BeautifulSoup
from utils._funcs import creating_club_dataframe

##### Club name and id

##### Funcs

In [96]:
"""
To make the request to the page we have to inform the
website that we are a browser and that is why we
use the headers variable
"""
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}

# def creating_club_dataframe(
#         club_name, 
#         club_id, 
#         headers
#     ):

#     # endereco_da_pagina stands for the data page address
#     endereco_da_pagina = "https://www.transfermarkt.com.br/"+club_name+"/startseite/verein/"+club_id+"?saison_id=2021"

#     # In the objeto_response variable we will the download of the web page
#     objeto_response = requests.get(endereco_da_pagina, headers=headers)

#     """
#     Now we will create a BeautifulSoup object from our object_response.
#     The 'html.parser' parameter represents which parser we will use when creating our object,
#     a parser is a software responsible for converting an entry to a data structure.
#     """
#     pagina_bs = BeautifulSoup(objeto_response.content, 'html.parser')

#     club_jog = [] # List that will receive all the players club - 

#     transfmkt_site = 'https://www.transfermarkt.com.br'

#     # Players names
#     nomes_jog = [] # List that will receive all the players names - check
#     tags_nomes = pagina_bs.find_all("span", {"class": "show-for-small"})
#     for tag_nome in tags_nomes:
#         nomes_jog.append(tag_nome.text)

#     # Players links
#     links_jog = [] # List that will receive all the players profile links - check
#     tags_links = BeautifulSoup(str(tags_nomes).strip('[]'), 'html.parser')
#     for a in tags_links.find_all('a', href=True):
#         links_jog.append(a['href'])

#     # Players Values links
#     links_values_jog = []
#     for link in links_jog:
#         links_values_jog.append(str(link).replace("profil","marktwertverlauf"))

#     # Players clubs
#     club_name_of = club_name.replace("-","_")
#     club_jog = [] # List that will receive all the players club - check
#     for c in range(len(nomes_jog)):
#         club_jog.append(club_name_of)

#     # Clubs championship
#     camp_jog = []
#     tags_camp = pagina_bs.find_all("span", {"class": "hauptpunkt"})
#     for tag_camp in tags_camp:
#         camp = str(tag_camp.text).replace(" ", "").replace("\n","")

#     for c in range(len(nomes_jog)):
#         camp_jog.append(camp)

#     # Players Positions
#     posicoes_jog = [] # List that will receive all the players positions - check
#     tags_goleiros = pagina_bs.find_all("td", {"class": "zentriert rueckennummer bg_Torwart"})
#     tags_zagueiros = pagina_bs.find_all("td", {"class": "zentriert rueckennummer bg_Abwehr"})
#     tags_meias = pagina_bs.find_all("td", {"class": "zentriert rueckennummer bg_Mittelfeld"})
#     tags_atacantes = pagina_bs.find_all("td", {"class": "zentriert rueckennummer bg_Sturm"})

#     # Gk positions
#     for c in range(len(tags_goleiros)):
#         posicoes_jog.append("Goleiro")

#     # Defenders positions
#     for c in range(len(tags_zagueiros)):
#         posicoes_jog.append("Defensor")

#     # Midfielders positions
#     for c in range(len(tags_meias)):
#         posicoes_jog.append("Meia")

#     # Strikers positions
#     for c in range(len(tags_atacantes)):
#         posicoes_jog.append("Atacante")

#     # Players age, heights, nacionalities
#     idade_jog = [] # List that will receive all the players age - check
#     alt_jog = [] # List that will receive all the players height - check
#     nac_jog = [] # List that will receive all the players nacionalities - check

#     for link in links_jog:
#         end = transfmkt_site + link
#         obj_resp = requests.get(end, headers=headers)
#         pag_bs = BeautifulSoup(obj_resp.content, 'html.parser')
#         tags_idades = pag_bs.find_all("span", {"itemprop": "birthDate"})
#         tags_nac = pag_bs.find_all("span", {"itemprop": "nationality"})
#         tags_alt = pag_bs.find_all("span", {"itemprop": "height"})
#         for tag_idade in tags_idades:
#             idade_jog.append(str(tag_idade.text).replace(" ", "")[12:14])

#         for tag_nac in tags_nac:
#             nac_jog.append(str(tag_nac.text).replace(" ", "")[1:])

#         for tag_alt in tags_alt:
#             alt_jog.append(float(str(tag_alt.text).replace(",",".")[:5].rstrip()))

#     # Players performance
#     gols_jog = []
    
    
    
    
    
    
    
#     # Players value
#     valor_jog = []


    
#     dic = {
#         'nome':nomes_jog,
#         'clube_atual':club_jog,
#         'campeonato':camp_jog,
#         'posicao':posicoes_jog,
#         'idade':idade_jog,
#         'nacionalidade':nac_jog,
#         'altura':alt_jog
#     }

#     df = pd.DataFrame(dic)

#     # Players transfers links
#     link_transfers_jog = []
#     for link in links_values_jog:
#         link_transfers_jog.append(link.replace('marktwertverlauf','transfers'))

#     # Players Seasons
#     temps_all = []
#     for link in link_transfers_jog:
#         end = transfmkt_site + link
#         obj_resp = requests.get(end, headers=headers)
#         pag_bs = BeautifulSoup(obj_resp.content, 'html.parser')
#         tags_seasons = pag_bs.find_all("div", {"class": "tm-player-transfer-history-grid__season"})
#         for tag in tags_seasons:
#             temps_all.append(str(tag.text).replace(" ", "").replace("\n",""))

#     cont = temps_all.count('Temporada')
#     idxs = []
#     c = 0
#     temps_jog = []

#     for d in range(0,cont):
#         i = temps_all.index('Temporada',c)
#         c = i + 1
#         idxs.append(i)
#         c = i + 1
#     idxs.append(len(temps_all))
#     idxs.pop(0)

#     c = 0
#     for d in range(0,cont):
#         part = temps_all[c:idxs[d]]
#         temps_jog.append(part)
#         c += len(part)

#     for lst in temps_jog:
#         lst.pop(0)

#     for lst in temps_jog:
#         if len(lst) == 0:
#             lst.append('22/23')
#         else:
#             pass

    
#     return temps_jog

In [101]:
lst1 = [1,3,3,1,3,3,3,1,3,1,3,3,3,3,3,3,1,1,3,3]
cont = lst1.count(1)
idxs = []
c = 0
lst2 = []

for d in range(0,cont):
    i = lst1.index(1,c)
    c = i + 1
    idxs.append(i)
    c = i + 1
idxs.append(len(lst1))
idxs.pop(0)

c = 0
for d in range(0,cont):
    part = lst1[c:idxs[d]]
    lst2.append(part)
    c += len(part)

for lst in lst2:
    lst.pop(0)

for lst in lst2:
    if len(lst) == 0:
        lst.append(3)
    else:
        pass

temp1_jog = []


lst2

[[3, 3], [3, 3, 3], [3], [3, 3, 3, 3, 3, 3], [3], [3, 3]]

In [100]:
"""
To make the request to the page we have to inform the
website that we are a browser and that is why we
use the headers variable
"""
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}

df = creating_club_dataframe('se-palmeiras','1023', headers)
df

[['17/18',
  '11/12',
  '09/10',
  '09/10',
  '09/10',
  '08/09',
  '08/09',
  '08/09',
  '07/08',
  '06/07',
  '05/06'],
 ['21/22', '16/17', '15/16', '14/15', '11/12', '11/12', '10/11'],
 ['19/20', '18/19', '18/19', '17/18', '14/15'],
 ['20/21', '19/20', '18/19', '16/17', '14/15', '11/12'],
 ['21/22', '19/20', '16/17'],
 ['16/17', '13/14'],
 ['19/20'],
 ['20/21', '14/15', '14/15', '14/15'],
 ['21/22', '19/20', '19/20', '16/17'],
 ['21/22',
  '20/21',
  '20/21',
  '19/20',
  '18/19',
  '18/19',
  '18/19',
  '16/17',
  '14/15'],
 ['22/23'],
 ['18/19',
  '18/19',
  '17/18',
  '11/12',
  '09/10',
  '09/10',
  '09/10',
  '07/08',
  '07/08',
  '07/08'],
 ['18/19', '18/19', '16/17', '12/13'],
 ['22/23', '19/20'],
 ['20/21', '20/21', '18/19'],
 ['21/22', '20/21', '18/19', '15/16', '15/16'],
 ['21/22', '19/20'],
 ['19/20'],
 ['18/19', '16/17', '16/17', '15/16', '15/16', '14/15', '13/14', '13/14'],
 ['21/22', '18/19', '18/19', '17/18'],
 ['18/19', '17/18', '16/17'],
 ['17/18', '17/18', '17/18',

In [2]:
"""
To make the request to the page we have to inform the
website that we are a browser and that is why we
use the headers variable
"""
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}

# endereco_da_pagina stands for the data page address
endereco_da_pagina = "https://www.transfermarkt.com.br/america-mineiro/startseite/verein/2863?saison_id=2021"

# In the objeto_response variable we will the download of the web page
objeto_response = requests.get(endereco_da_pagina, headers=headers)

"""
Now we will create a BeautifulSoup object from our object_response.
The 'html.parser' parameter represents which parser we will use when creating our object,
a parser is a software responsible for converting an entry to a data structure.
"""
pagina_bs = BeautifulSoup(objeto_response.content, 'html.parser')

In [222]:
club_jog = [] # List that will receive all the players club - 

transfmkt_site = 'https://www.transfermarkt.com.br'

# Players names
nomes_jog = [] # List that will receive all the players names - check
tags_nomes = pagina_bs.find_all("span", {"class": "show-for-small"})
for tag_nome in tags_nomes:
    nomes_jog.append(tag_nome.text)

# Players links
links_jog = [] # List that will receive all the players profile links - check
tags_links = BeautifulSoup(str(tags_nomes).strip('[]'), 'html.parser')
for a in tags_links.find_all('a', href=True):
    links_jog.append(a['href'])

# Players Values links
links_values_jog = []
for link in links_jog:
    links_values_jog.append(str(link).replace("profil","marktwertverlauf"))


# Clubs championship
camp_jog = []
tags_camp = pagina_bs.find_all("span", {"class": "hauptpunkt"})
for tag_camp in tags_camp:
    camp = str(tag_camp.text).replace(" ", "").replace("\n","")

for c in range(len(nomes_jog)):
    camp_jog.append(camp)

# Players Positions
posicoes_jog = [] # List that will receive all the players positions - check
tags_goleiros = pagina_bs.find_all("td", {"class": "zentriert rueckennummer bg_Torwart"})
tags_zagueiros = pagina_bs.find_all("td", {"class": "zentriert rueckennummer bg_Abwehr"})
tags_meias = pagina_bs.find_all("td", {"class": "zentriert rueckennummer bg_Mittelfeld"})
tags_atacantes = pagina_bs.find_all("td", {"class": "zentriert rueckennummer bg_Sturm"})

# Gk positions
for c in range(len(tags_goleiros)):
    posicoes_jog.append("Goleiro")

# Defenders positions
for c in range(len(tags_zagueiros)):
    posicoes_jog.append("Defensor")

# Midfielders positions
for c in range(len(tags_meias)):
    posicoes_jog.append("Meia")

# Strikers positions
for c in range(len(tags_atacantes)):
    posicoes_jog.append("Atacante")

# # Players age, heights, nacionalities
# idade_jog = [] # List that will receive all the players age - check
# alt_jog = [] # List that will receive all the players height - check
# nac_jog = [] # List that will receive all the players nacionalities - check

# for link in links_jog:
#     end = transfmkt_site + link
#     obj_resp = requests.get(end, headers=headers)
#     pag_bs = BeautifulSoup(obj_resp.content, 'html.parser')
#     tags_idades = pag_bs.find_all("span", {"itemprop": "birthDate"})
#     tags_nac = pag_bs.find_all("span", {"itemprop": "nationality"})
#     tags_alt = pag_bs.find_all("span", {"itemprop": "height"})
#     for tag_idade in tags_idades:
#         idade_jog.append(str(tag_idade.text).replace(" ", "")[12:14])

#     for tag_nac in tags_nac:
#         nac_jog.append(str(tag_nac.text).replace(" ", "")[1:])

#     for tag_alt in tags_alt:
#         alt_jog.append(float(str(tag_alt.text).replace(",",".")[:5].rstrip()))


In [224]:
# print(nomes_jog, len(nomes_jog))
# print(posicoes_jog, len(posicoes_jog))
# print(idade_jog, len(idade_jog))
# print(nac_jog, len(nac_jog))
# print(alt_jog, len(alt_jog))
links_values_jog

['/jori/marktwertverlauf/spieler/429799',
 '/airton/marktwertverlauf/spieler/377799',
 '/matheus-cavichioli/marktwertverlauf/spieler/54274',
 '/robson/marktwertverlauf/spieler/902033',
 '/german-conti/marktwertverlauf/spieler/284321',
 '/iago-maidana/marktwertverlauf/spieler/348261',
 '/eder/marktwertverlauf/spieler/375690',
 '/luan-patrick/marktwertverlauf/spieler/658674',
 '/ricardo-silva/marktwertverlauf/spieler/206097',
 '/marlon-lopes/marktwertverlauf/spieler/443721',
 '/danilo-avelar/marktwertverlauf/spieler/140748',
 '/joao-paulo/marktwertverlauf/spieler/167471',
 '/patric/marktwertverlauf/spieler/95087',
 '/raul-caceres/marktwertverlauf/spieler/147410',
 '/ze-ricardo/marktwertverlauf/spieler/476336',
 '/lucas-kal/marktwertverlauf/spieler/435489',
 '/juninho/marktwertverlauf/spieler/271187',
 '/flavio/marktwertverlauf/spieler/672618',
 '/gustavinho/marktwertverlauf/spieler/800649',
 '/martin-benitez/marktwertverlauf/spieler/189418',
 '/juan-pablo-ramirez/marktwertverlauf/spieler

### Gráfico interativo

In [2]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver
import chromedriver_binary
import datetime as dt
import pandas as pd

# Opening the connection and grabbing the page
my_url = 'https://www.google.com/webhp?hl=en'
option = Options()
option.headless = True

In [3]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(800, 800))  
display.start()
driver = webdriver.Chrome(executable_path='/home/gabriel/.local/bin/chromedriver',options=option)
driver.get(my_url)
driver.maximize_window()

FileNotFoundError: [Errno 2] No such file or directory: 'Xvfb'

In [2]:
import os
import shutil
import tempfile
import time
import undetected_chromedriver as webdriver

def force_patcher_to_use(directory):
    # copy the chromedriver in directory
    exe = webdriver.Patcher.exe_name
    executable_path = os.path.join(directory, exe)
    src = os.path.join(webdriver.Patcher.data_path, exe)
    if os.path.exists(src):
        shutil.copyfile(src, executable_path)

    # monkey patch the Patcher class
    class PatcherWithForcedExecutablePath(webdriver.Patcher):
        def __init__(self, *args, **kwargs):
            kwargs["executable_path"] = executable_path
            super().__init__(*args, **kwargs)

            # download the chromedriver if it was not already done above
            if not os.path.exists(executable_path):
                release = self.fetch_release_number()
                self.version_main = release.version[0]
                self.version_full = release
                self.unzip_package(self.fetch_package())

    webdriver.Patcher = PatcherWithForcedExecutablePath
    return executable_path

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
force_patcher_to_use('/home/gabriel/.local/lib/python3.10/site-packages/chromedriver_binary')

'/home/gabriel/.local/lib/python3.10/site-packages/chromedriver_binary/chromedriver'